In [1]:
import cv2
import numpy as np
import os
import random

# Parametri dataset
num_images = 5000        # numero di immagini da generare
img_size = 32             # dimensione 32x32
min_shapes = 1            # numero minimo di forme per immagine
max_shapes = 4            # numero massimo di forme per immagine
num_classes = 5           # numero di classi (da 1 a num_classes, 0 riserva lo sfondo)

# Cartelle di output aggiornate
output_dirs = {
    'rgb': 'data/rgb_images',
    'label_png': 'data/labels_png',  # Ex "labels"
    'depth': 'data/depth_maps',
    'label_npy': 'data/labels'  # Ex "labels_npy"
}

for dir_path in output_dirs.values():
    os.makedirs(dir_path, exist_ok=True)

# Funzioni per disegnare le forme
def draw_random_circle(img, label_img, depth_img, label_value, depth_value):
    center = (random.randint(0, img_size-1), random.randint(0, img_size-1))
    radius = random.randint(3, img_size//3)
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.circle(img, center, radius, color, -1)
    cv2.circle(label_img, center, radius, label_value, -1)
    cv2.circle(depth_img, center, radius, depth_value, -1)

def draw_random_rectangle(img, label_img, depth_img, label_value, depth_value):
    pt1 = (random.randint(0, img_size-10), random.randint(0, img_size-10))
    pt2 = (min(pt1[0] + random.randint(5, img_size//2), img_size-1),
           min(pt1[1] + random.randint(5, img_size//2), img_size-1))
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.rectangle(img, pt1, pt2, color, -1)
    cv2.rectangle(label_img, pt1, pt2, label_value, -1)
    cv2.rectangle(depth_img, pt1, pt2, depth_value, -1)

def draw_random_triangle(img, label_img, depth_img, label_value, depth_value):
    pts = np.array([
        [random.randint(0, img_size-1), random.randint(0, img_size-1)],
        [random.randint(0, img_size-1), random.randint(0, img_size-1)],
        [random.randint(0, img_size-1), random.randint(0, img_size-1)]
    ])
    color = [random.randint(0, 255) for _ in range(3)]
    cv2.fillPoly(img, [pts], color)
    cv2.fillPoly(label_img, [pts], label_value)
    cv2.fillPoly(depth_img, [pts], depth_value)

# Generazione dataset
for i in range(num_images):
    # Inizializzo le immagini vuote:
    # rgb: immagine a 3 canali, inizialmente nera
    rgb_img = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    # label: immagine a 1 canale; 0 indica lo sfondo
    label_img = np.zeros((img_size, img_size), dtype=np.uint8)
    # depth: immagine a 1 canale; valori in scala 0-255
    depth_img = np.zeros((img_size, img_size), dtype=np.uint8)
    
    # Numero casuale di forme da disegnare
    num_shapes = random.randint(min_shapes, max_shapes)
    for _ in range(num_shapes):
        # Seleziono in maniera casuale la forma da disegnare
        shape_type = random.choice(['circle', 'rectangle', 'triangle'])
        # Seleziono la classe (da 1 a num_classes)
        label_value = random.randint(1, num_classes)
        # Per la profondità assegno un valore casuale (evito valori troppo bassi per distinguere lo sfondo)
        depth_value = random.randint(50, 255)
        
        if shape_type == 'circle':
            draw_random_circle(rgb_img, label_img, depth_img, label_value, depth_value)
        elif shape_type == 'rectangle':
            draw_random_rectangle(rgb_img, label_img, depth_img, label_value, depth_value)
        elif shape_type == 'triangle':
            draw_random_triangle(rgb_img, label_img, depth_img, label_value, depth_value)
    
    # Salvo le immagini nei rispettivi file con il nuovo naming:
    rgb_filename = os.path.join(output_dirs['rgb'], f"rgb_{i}.png")
    label_png_filename = os.path.join(output_dirs['label_png'], f"label_{i}.png")  # Ex "labels"
    depth_filename = os.path.join(output_dirs['depth'], f"depth_{i}.png")
    
    cv2.imwrite(rgb_filename, rgb_img)
    cv2.imwrite(label_png_filename, label_img)
    cv2.imwrite(depth_filename, depth_img)
    
    # Salvo l'array NumPy delle etichette nella cartella labels (ex labels_npy)
    npy_filename = os.path.join(output_dirs['label_npy'], f"label_{i}.npy")
    np.save(npy_filename, label_img)

print(f"Dataset generato: {num_images} immagini in {os.path.abspath('data')}")

Dataset generato: 5000 immagini in /Users/ivan/Desktop/AP4Fed/dataset/iv4N/data
